I expanded on DataCanary's analysis on best & worst NBA defenders during the 2014-15 season.

Instead of looking at pure shots made and shots missed "in a defender's face," I looked at the **difference between a shooter's normal FG% and their FG% when each defender guarded them**.

*brief set-up code*

In [ ]:
import pandas as pd
import numpy as np
import pylab as P
%matplotlib inline

In [ ]:
# import data
shot_df = pd.read_csv('../input/shot_logs.csv', header=0)

# defenders
defender_df = pd.concat([shot_df['CLOSEST_DEFENDER_PLAYER_ID'], shot_df['CLOSEST_DEFENDER']], axis=1, keys=['PLAYER_ID', 'PLAYER'])
defender_df = defender_df.drop_duplicates()

# shooters
shooter_df = pd.concat([shot_df['player_id'], shot_df['player_name']], axis=1, keys=['PLAYER_ID', 'PLAYER'])
shooter_df = shooter_df.drop_duplicates()

In [ ]:
# FG%
for index, row in shooter_df.iterrows():
    this_id = row['PLAYER_ID']
    
    shooter_df.loc[ (shooter_df['PLAYER_ID'] == this_id), 'FGM']\
        = shot_df[ (shot_df['SHOT_RESULT'] == 'made') 
            & (shot_df['player_id'] == this_id)]['player_id'].count()
    
    shooter_df.loc[ (shooter_df['PLAYER_ID'] == this_id), 'FGA']\
        = shot_df[(shot_df['player_id'] == this_id)]['player_id'].count()
    
    shooter_df['FG%'] = shooter_df['FGM'] / shooter_df['FGA']

In [ ]:
# DFG%
for index, row in defender_df.iterrows():
    this_id = row['PLAYER_ID']
    
    defender_df.loc[ (defender_df['PLAYER_ID'] == this_id), 'DFGM']\
        = shot_df[ (shot_df['SHOT_RESULT'] == 'made')\
          & (shot_df['CLOSEST_DEFENDER_PLAYER_ID'] == this_id)]['player_id'].count()
    
    defender_df.loc[ (defender_df['PLAYER_ID'] == this_id), 'DFGA']\
        = shot_df[(shot_df['CLOSEST_DEFENDER_PLAYER_ID'] == this_id)]['player_id'].count()
    
    defender_df['DFG%'] = defender_df['DFGM'] / defender_df['DFGA']
    
    # OFG% - average FG% of shooters this defender guards
    defender_dict = {}
    for shooter_index, shooter_row in shooter_df.iterrows():
        shooter_id = shooter_row['PLAYER_ID']
        shots_against_player = shot_df[ (shot_df['CLOSEST_DEFENDER_PLAYER_ID'] == this_id)\
                                        & (shot_df['player_id'] == shooter_id)]['player_id'].count()
        if shots_against_player > 0:
            defender_dict[shooter_id] = shots_against_player
    
    ofg = 0.0
    total_shots = defender_df[ (defender_df['PLAYER_ID'] == this_id)]['DFGA']
    for shooter, shots in defender_dict.items():
        ofg += shots/total_shots*shooter_df[ (shooter_df['PLAYER_ID'] == shooter)].iloc[0]['FG%']
    
    defender_df.loc[ (defender_df['PLAYER_ID'] == this_id), 'oFG%'] = ofg

In [ ]:
defender_df['diff'] = defender_df['oFG%'] - defender_df['DFG%']
diff_df = defender_df.sort_values(by='diff', axis=0, ascending=False, inplace=False)
diff_df.head(10)

#The Best

In [ ]:
diff_df[ (diff_df['DFGA'] > 300.0)].head(10)

After filtering for players who have defended at least 300 FGA, our best defender is **Andrew Bogut** with a whopping 9.1% defensive field goal improvement.

It's also interesting to look at the difference between the top 10 players. There are sharp drop offs between Allen/Roberson and Hibbert/Draymond.

Also interesting: Kawhi Leonard, the 2015 DPOY, doesn't show up (only a 1.5% improvement). 

In [ ]:
diff_df[ (diff_df['PLAYER'] == "Leonard, Kawhi")].head(1)

In [ ]:
#The Worst

In [ ]:
diff_df = defender_df.sort_values(by='diff', axis=0, ascending=True, inplace=False)
diff_df[ (diff_df['DFGA'] > 100.0)].head(10)

For our worst players I limited FGA defended to at least 100, since worse players tend to play less. The winner for worst defender goes to **Jose Calderon**!

In [ ]:
diff_df[ (diff_df['DFGA'] > 300.0)].head(10)

For players with over 300 FGs defended, the worst player is **Trey Burke**. Interestingly enough, **Trevor Ariza** shows up here. 

In [ ]:
Side note: my man **James Harden** is actually an above average defender. Wow.

In [ ]:
diff_df[ (diff_df['PLAYER'] == "Harden, James")].head(1)

#To be continued
There are a couple other factors I haven't considered yet, including distance from shooter, seconds left, dribbles before shot, etc. 

Perhaps Kawhi/Ariza are such great defenders they happen to be the closest to an open shooter, decreasing their DFG%. Likewise, Harden could be such an awful defender that he lucks out of open shots.